# Cellpose segmentation

Notebook showing how to segment cells using Cellpose.

In [12]:
import os
import numpy as np

from nd2reader import ND2Reader
from tifffile import imread,TiffFile
from skimage.io import imsave

from skimage.io import imread as imread_png

from cellpose import models

import napari
from skimage.filters import gaussian

In [2]:
# load cellpose model
model = models.Cellpose(gpu=False, model_type='cyto')

2021-09-02 13:14:13,839 [INFO] >>>> using CPU


In [3]:
im_dir = r'X:\Sonja\LPC Confocal\DE confocal 20x 24wp 1mil-4mil001 pt2'
im_file = r'38_LPC_1mil_1.4_test.tif'

save_dir = r'X:\Sonja\LPC Confocal\test'

In [8]:
# check how many timepoints are there in the file
t=TiffFile(os.path.join(im_dir,im_file))

frames_num = len(t.pages)

print(f'Total frame number: {frames_num}')

Total frame number: 3


In [10]:
# loop for segmentation 

#for i in range(frames_num): # use this line to segment all frames
for i in range(0,frames_num): # it's a small example - just 4 first frames
    
    # get an image
    im = imread(os.path.join(im_dir,im_file),key=i)

    # segment the right plane
    labels, _, _, _ = model.eval(im, diameter=15, channels=[0,0])

    # save segmentation
    save_path = os.path.join(save_dir,im_file.replace('.tif',f'_{str(i).zfill(3)}.png'))
    imsave(save_path,labels.astype('uint16')) 

2021-09-02 13:28:25,969 [INFO] ~~~ FINDING MASKS ~~~
2021-09-02 13:39:10,670 [INFO] >>>> TOTAL TIME 644.70 sec


C:\Users\kmkedz\AppData\Local\Temp/ipykernel_25892/3521982086.py:14: UserWarning: X:\Sonja\LPC Confocal\test\38_LPC_1mil_1.4_test_000.png is a low contrast image
  imsave(save_path,labels.astype('uint16'))


2021-09-02 13:39:11,113 [INFO] ~~~ FINDING MASKS ~~~
2021-09-02 13:49:51,184 [INFO] >>>> TOTAL TIME 640.07 sec
2021-09-02 13:49:51,401 [INFO] ~~~ FINDING MASKS ~~~


C:\Users\kmkedz\AppData\Local\Temp/ipykernel_25892/3521982086.py:14: UserWarning: X:\Sonja\LPC Confocal\test\38_LPC_1mil_1.4_test_001.png is a low contrast image
  imsave(save_path,labels.astype('uint16'))


KeyboardInterrupt: 

In [8]:
save_path = os.path.join(save_dir,im_file.replace('.nd2',f'_{str(i).zfill(3)}.png'))
imsave(save_path,labels.astype('uint16')) 

C:\Users\martjohn\AppData\Local\Temp/ipykernel_17484/2968871168.py:2: UserWarning: E:\Martha\2021Imaging\08242021p27\Segmentation Test MJ\01_288.png is a low contrast image
  imsave(save_path,labels.astype('uint16'))


## Visualize results

In [13]:
# we need to read in both images and masks now
# for segmentation we were keeping only one mask at the time

# empty lists as containers
all_im = []
all_masks = []

for i in range(2):
    
    # get an image
    im = imread(os.path.join(im_dir,im_file),key=i)
    
    # store the image in the list
    all_im.append(im)
    
    # get the mask
    mask_path = os.path.join(save_dir,im_file.replace('.tif',f'_{str(i).zfill(3)}.png'))
    mask = imread_png(mask_path)
    
    # store the mask in the list
    all_masks.append(mask)

# change the format
all_im = np.array(all_im)
all_masks = np.array(all_masks)

In [ ]:
# visualize
viewer = napari.Viewer()
viewer.add_image(all_im,blending='additive')
viewer.add_labels(all_masks,opacity = 0.5)

## Example - segmentation for all the files in a given directory

In [ ]:
im_dir = r'Z:\COOK_LAB\CELL_TRACKING\example_images'
im_file = r'01.nd2'

save_dir = r'Z:\COOK_LAB\CELL_TRACKING\example_results'

In [36]:
# main loop over all the files in the directory
for my_file in os.listdir(im_dir):
    
    nd2_movie = ND2Reader(os.path.join(im_dir,im_file))
    frames_num = nd2_movie.sizes['t']
    
    # internal loop through all the frames in each movie
    for i in range(frames_num):
   
        # get an image
        im = nd2_movie.get_frame_2D(c=0, t=i)

        # segment the right plane
        labels, _, _, _ = model.eval(im, diameter=30, channels=[[0, ]])

        # save segmentation
        save_path = os.path.join(save_dir,my_file.replace('.nd2',f'_{str(i).zfill(3)}.tif'))
        imsave(save_path,labels.astype('uint16'))

['01.nd2',
 '02.nd2',
 '03.nd2',
 '04.nd2',
 '05.nd2',
 '06.nd2',
 '07.nd2',
 '08.nd2',
 '09.nd2',
 '10.nd2',
 '11.nd2',
 '12.nd2',
 '13.nd2',
 '14.nd2',
 '15.nd2']

## Example - preprocessing to improve segmentation

In [28]:
# in this particular movie PCNA pattern is not a problem for detection (but it was observed in the past)
# segmentation with smoothing

# loop for segmentation 

#for i in range(frames_num): # use this line to segment all frames
for i in range(4): # it's a small example - just 4 first frames
    
    # get an image
    im = nd2_movie.get_frame_2D(c=0, t=i)
    
    # smooth the image
    im = gaussian(im, sigma=2)

    # segment the right plane
    labels, _, _, _ = model.eval(im, diameter=30, channels=[0,0])

    # save segmentation
    save_path = os.path.join(save_dir,im_file.replace('.nd2',f'_{str(i).zfill(3)}_smooth.tif'))
    imsave(save_path,labels.astype('uint16')) 

2021-08-25 15:18:54,123 [INFO] ~~~ FINDING MASKS ~~~
2021-08-25 15:20:25,430 [INFO] >>>> TOTAL TIME 91.31 sec
2021-08-25 15:20:25,729 [INFO] ~~~ FINDING MASKS ~~~
2021-08-25 15:21:55,866 [INFO] >>>> TOTAL TIME 90.14 sec
2021-08-25 15:21:56,169 [INFO] ~~~ FINDING MASKS ~~~
2021-08-25 15:23:26,696 [INFO] >>>> TOTAL TIME 90.53 sec
2021-08-25 15:23:26,981 [INFO] ~~~ FINDING MASKS ~~~
2021-08-25 15:24:56,787 [INFO] >>>> TOTAL TIME 89.81 sec


In [30]:
# we need to read in both images and masks now
# for segmentation we were keeping only one mask at the time

# empty lists as containers
all_masks_smooth = []

for i in range(4):
    
    # get the mask
    mask_path = os.path.join(save_dir,im_file.replace('.nd2',f'_{str(i).zfill(3)}.tif'))
    mask = imread(mask_path)
    
    # store the mask in the list
    all_masks_smooth.append(mask)

# change the format
all_masks_smooth = np.array(all_masks_smooth)

# prepare smooth images for visualizatin
all_im_smooth = gaussian(all_im, sigma=2)

In [31]:
# visualize
viewer = napari.Viewer()
viewer.add_image(all_im,blending='additive')
viewer.add_image(all_im_smooth,blending='additive')
viewer.add_labels(all_masks,opacity = 0.5)

C:\ProgramData\Anaconda3\envs\cellpose\lib\site-packages\napari\_vispy\vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


<Labels layer 'all_masks' at 0x2e75b8b7f08>